Code to read survey results

Number of participants by region and language used.
The type of sample method used in the survey (simple, complex, etc.).
What is the survey representative of when you use the sample weights and when you don’t use the sample weights. This could be country, household, or individual.
The questions you would like to use in the thesis together with some descriptive statistics (for this you can omit considering the sample design and weights):
type of data: dichotomous; categorical (which categories); continuous (which range); open question (which language).
Correlation analysis

In [1]:
import pandas as pd
import numpy as np
import openpyxl

In [2]:
survey_questions  = pd.read_csv('ESS_files\ESS10.csv')
# Clean the survey questions dataframe to make it more usable for mapping
survey_questions

C:\Users\coolg\AppData\Local\Temp\ipykernel_23380\3705931749.py:1: DtypeWarning: Columns (166,172,174,607,608) have mixed types. Specify dtype option on import or set low_memory=False.
  survey_questions  = pd.read_csv('ESS_files\ESS10.csv')


,name,essround,edition,proddate,idno,cntry,dweight,pspwght,pweight,anweight,...,vinwe,inwde,jinws,jinwe,inwtm,mode,domain,prob,stratum,psu
0,ESS10e03_2,10,3.2,02.11.2023,10038,BE,0.882220,0.972276,0.718075,0.698167,...,2022-09-01 17:47:00,2022-09-01 17:47:00,2022-09-01 17:47:00,2022-09-01 17:47:00,36.0,1,1.0,0.000397,188,2596
1,ESS10e03_2,10,3.2,02.11.2023,10053,BE,1.047643,0.888635,0.718075,0.638107,...,2022-04-08 11:07:00,2022-04-08 11:10:00,2022-04-08 11:07:00,2022-04-08 11:10:00,54.0,2,2.0,0.000334,194,2206
2,ESS10e03_2,10,3.2,02.11.2023,10055,BE,1.087741,0.722811,0.718075,0.519033,...,2022-05-20 11:08:00,2022-05-20 11:10:00,2022-05-20 11:08:00,2022-05-20 11:10:00,77.0,1,2.0,0.000322,198,2114
3,ESS10e03_2,10,3.2,02.11.2023,10062,BE,0.909910,1.005565,0.718075,0.722072,...,2022-05-22 13:58:00,2022-05-22 13:59:00,2022-05-22 13:58:00,2022-05-22 13:59:00,55.0,1,1.0,0.000385,150,2645
4,ESS10e03_2,10,3.2,02.11.2023,10064,BE,0.918949,0.638705,0.718075,0.458639,...,2022-05-18 11:44:00,2022-05-18 11:45:00,2022-05-18 11:44:00,2022-05-18 11:45:00,55.0,1,1.0,0.000381,149,2313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37606,ESS10e03_2,10,3.2,02.11.2023,27808,SK,0.515714,0.339385,0.323800,0.109893,...,2021-06-08 14:28:34,2021-06-08 14:30:41,2021-06-08 14:29:01,2021-06-08 14:31:44,70.0,1,1.0,0.001522,2610,27206
37607,ESS10e03_2,10,3.2,02.11.2023,27826,SK,0.297974,0.196093,0.323800,0.063495,...,2021-08-02 10:33:27,2021-08-02 10:36:27,2021-08-02 10:35:22,2021-08-02 10:37:34,45.0,1,2.0,0.002635,2610,27217
37608,ESS10e03_2,10,3.2,02.11.2023,27834,SK,0.965931,0.857000,0.323800,0.277497,...,2021-06-26 20:52:15,2021-06-26 20:53:05,2021-06-26 20:52:27,2021-06-26 20:54:32,33.0,1,1.0,0.000813,2631,27134
37609,ESS10e03_2,10,3.2,02.11.2023,27846,SK,0.854279,0.624287,0.323800,0.202144,...,2021-07-21 14:14:41,2021-07-21 14:17:31,2021-07-21 14:16:38,2021-07-21 14:18:38,43.0,1,1.0,0.000919,2638,27183


In [9]:
from bs4 import BeautifulSoup
import pandas as pd

# Define the path to the HTML file
html_file_path = 'ESS_files/ESS10 codebook.html'

# Initialize a list to hold the rows of the DataFrame
rows = []

# Read the HTML file
with open(html_file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content
soup = BeautifulSoup(html_content, 'lxml')

# Find all <h3> tags with an 'id' attribute (these contain the questions)
question_tags = soup.find_all('h3', id=True)

# Iterate over the question tags to extract the details
for tag in question_tags:
    idnumber = tag.get('id')  # Get the ID of the question
    questiontext = tag.find_next_sibling('div').text.strip()  # Get the question text
    
    # Find the next div that possibly contains the table
    table_container = tag.find_next_sibling('div').find_next_sibling('div')
    if table_container:
        table = table_container.find('table')
    else:
        table = None

    # Initialize an empty list to hold the response options
    responseoptions = []

    # If a table is found, extract options
    if table:
        for row in table.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) == 2:  # Ensure exactly 2 cells are found (value and description)
                value = cells[0].text.strip()
                description = cells[1].text.strip()
                responseoptions.append(f"{value}: {description}")

    # Convert the list of response options to a string
    responseoptions_str = "; ".join(responseoptions)

    # Append the extracted information as a row to the rows list
    rows.append({
        'id': idnumber,
        'questiontext': questiontext,
        'responseoptions': responseoptions_str
    })

# Create a DataFrame from the rows list
df = pd.DataFrame(rows)


     idnumber                        questiontext  \
0        name                    Title of dataset   
1    essround                           ESS round   
2     edition                             Edition   
3    proddate                     Production date   
4        idno  Respondent's identification number   
..        ...                                 ...   
613      mode             Mode of data collection   
614    domain                     Sampling domain   
615      prob                Sampling probability   
616   stratum                    Sampling stratum   
617       psu               Primary sampling unit   

                                       responseoptions  
0                                                       
1                                                       
2                                                       
3                                                       
4                                                       
..                   

In [11]:
df

,idnumber,questiontext,responseoptions
0,name,Title of dataset,
1,essround,ESS round,
2,edition,Edition,
3,proddate,Production date,
4,idno,Respondent's identification number,
...,...,...,...
613,mode,Mode of data collection,"1: Interview, Face to face (CAPI); 2: Intervie..."
614,domain,Sampling domain,
615,prob,Sampling probability,
616,stratum,Sampling stratum,


In [12]:
survey_questions['stratum']

0         188
1         194
2         198
3         150
4         149
         ... 
37606    2610
37607    2610
37608    2631
37609    2638
37610    2620
Name: stratum, Length: 37611, dtype: int64